<a href="https://colab.research.google.com/github/tavi1402/Data_Science_bootcamp/blob/main/Assignment_advanced_sql_querying_practice_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment - Advanced SQL Querying Practice

As you go through this notebook, you will find the symbol **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end without errors.

**Guidelines**

1. Make sure to run all the code cells in order. Otherwise, you may get errors like `NameError` for undefined variables.
2. Do not change variable names, delete cells, or disturb other existing code. It may cause problems during evaluation.
3. In some cases, you may need to add some code cells or new statements before or after the line of code containing the **???**.
4. Since you'll be using a temporary online service for code execution, save your work by pressing **cntrl+ s or cmd + s** at regular intervals.
5. Review the "Evaluation Criteria" for the assignment carefully and make sure your submission meets all the criteria.
7. It's okay to ask for help & discuss ideas on the discussions, but please don't post the full working code, to give everyone an opportunity to solve the assignment on their own.
8. **Please make sure that the column names are same as in the screenshot under each question.**


## How to Run the Code and Save Your Work


**Option 1: Running using free online resources (1-click, recommended):** The easiest way to start executing the code is to click the **Run** button at the top of this page and select **Run on Binder**. This will set up a cloud-based Jupyter notebook server and allow you to modify/execute the code.


**Option 2: Running on your computer locally:** To run the code on your computer locally, you'll need to set up [Python](https://www.python.org), download the notebook and install the required libraries. Click the **Run** button at the top of this page, select the **Run Locally** option, and follow the instructions.

## SQLite and Initial Setup

Relational databases generally have two components:

1. **Database Server/Engine**: A software package that manages databases and runs in the background, listening for SQL queries from authorized users E.g. MySQL server, Microsoft SQL server, Postgres etc.
2. **Database Client**: A command-line tool or graphical user interface (GUI) to connect to the database server and run SQL queries. E.g. MySQL workbench, PgAdmin etc.

The server and client can be on the same computer e.g. both on your laptop, or on different computers e.g. the database server can be running on the cloud and you can connect to it using a client installed on your computer.

Most database servers/engines are designed to operate on databases containing large amounts of data (e.g. 100s of GBs) and to handle a very high volume of queries (e.g. thousands of queries per second). They typically require powerful hardware i.e. multi-core CPUs and large amounts of RAM.


<img src="https://www.sqlite.org/images/sqlite370_banner.gif" width="240" style="margin-left:0">

In this assignment, however, we'll use a lightweight database engine called [SQLite](https://www.sqlite.org/index.html), which is well-suited for databases with small amounts of data and has very limited CPU & RAM requirements. Despite being limited in its capabilities, it is the [most widely used database engine in the world](https://www.sqlite.org/mostdeployed.html). SQLite is used by smartphone apps, web browsers, desktop applications, IoT devices etc. to store and manage data locally on the device.

If you're running this assignment locally, you'll need to [download and install `sqlite3`](https://www.servermania.com/kb/articles/install-sqlite/) on your computer. `sqlite3` is already installed on Binder. You can verify that you have `sqlite3` installed by running the following command to show the version of SQLite you have installed:

In [ ]:
!sqlite3 --version

3.36.0 2021-06-18 18:36:39 5c9a6c06871cb9fe42814af9c039eb6da5427a6ec28f187af7ebfb62eafa66e5


Unlike other relational databases, SQLite doesn't have separate server and client packages. The `sqlite3` command line tool is all your need to create and interact with SQLite databases. The databases themselves are stored as files with the extension `.sqlite`. You can perform CRUD operations on the database simply by passing SQL queries using `sqlite3`.

Here's a visual representation of how SQLite differs from other relational database servers ([source](https://devopedia.org/sqlite)):

<img src="https://i.imgur.com/eC5Ieni.png" width="640">


Note that a `.sqlite` file is different from a `.sql` file, which contains commands for creating tables and inserting data. The `.sqlite` file is the actual database where the data is stored in a binary tabular format for efficient querying and manipulation.

In this assignment, we'll use the [Chinook open source database](https://github.com/lerocha/chinook-database). Let's begin by downloading the `.sqlite` file for the database containing all the required tables and the sample data.

In [ ]:
from urllib.request import urlretrieve

In [ ]:
db_url = 'https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite'

In [ ]:
urlretrieve(db_url, 'chinook.sqlite')

('chinook.sqlite', <http.client.HTTPMessage at 0x7f50c0725550>)

The database `chinook.sqlite` is now downloaded. To access and interact with the database by writing SQL queries directly within Jupyter, we'll use the [`ipython-sql`](https://pypi.org/project/ipython-sql/) library that provides the `%%sql` magic commands. It uses the [`sqlalchemy`](https://sqlalchemy.org) library behind the scenes to interact with the database.

In [ ]:
!pip install ipython-sql --quiet --upgrade

In [ ]:
!pip install sqlalchemy --quiet

In [ ]:
%load_ext sql

We can now connect to the database using a [SQLAlchemy connection string](https://docs.sqlalchemy.org/en/14/core/engines.html). We'll use the `%%sql` Jupyter magic command.

In [ ]:
%%sql

sqlite:///chinook.sqlite

We are now connected to the database and we can start writing SQL queries.

## Chinook Database Structure and Queries

The Chinook database represents a digital media store, including tables for artists, albums, media tracks, invoices and customers. Here's an [Entity Relationship Diagram](https://www.guru99.com/er-diagram-tutorial-dbms.html) (ERD) showing the structure of the Chinook database:

![](https://i.imgur.com/X1wM142.png)

Let's begin by looking at the data from some of the tables in the database. We can write SQL queries directly within Jupyter code cells by including the magic command `%%sql` as the first line of the cell, indicating that contents of cell represent a SQL query.

In [ ]:
%%sql

SELECT * FROM Artist LIMIT 5

 * sqlite:///chinook.sqlite
Done.


ArtistId,Name
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


In [ ]:
%%sql

SELECT * FROM Album LIMIT 5

 * sqlite:///chinook.sqlite
Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


## Window Function Syntax

> **QUESTION 1**: Write a SQL query to give the tracks in every album sequential track numbers. If an album has 9 tracks, the track number should be from 1 to 9. The output should display albumid, trackname, trackid, tracknumber.

![](https://i.imgur.com/PAS6vg2.png)

In [ ]:
%%sql

???

In [ ]:
# USED FOR EVALUATION. DON'T MODIFY/DELETE/MOVE THIS CELL!
ans1 = _

## Ranking Functions

> **QUESTION 2**: Write a SQL query to rank all the albums based on highest number of tracks. The output should display albumid, album title, TotalTracks & AlbumRank.

*Note:* If two albums have same number of total tracks, they should be assigned the same rank and the consecutive rank must be skipped. Please see the expected output for better understanding.

![](https://i.imgur.com/JLpFhCn.png)

In [ ]:
%%sql

???

In [ ]:
# DON'T MODIFY THIS CELL! IT IS USED FOR EVALUATION.
ans2 = _

## Date Functions

> **QUESTION 3**: Write a SQL query to list employees hired after the year 2001 & before the year 2003 with their full names, employeeId & the date of hiring.
>
> *Note*: SQLite doesn't support the `YEAR` function. Instead use `strftime("%Y", Invoice.InvoiceDate)` to extract the year from the column `InvoiceDate` as a string. [Learn more.](https://www.w3resource.com/sqlite/sqlite-strftime.php)
>
> _Hint_: First try to write down a step-by-step solution to the problem in plain English, and then try to convert it to a SQL query. Use the empty cells below to experiment with intermediate queries.

![](https://i.imgur.com/lFMJBA3.png)

In [ ]:
%%sql

???

In [ ]:
# DON'T MODIFY THIS CELL! IT IS USED FOR EVALUATION.
ans3 = _

> **QUESTION 4**: Write a SQL query to show the invoice id, invoice date, year, month number, and month day for invoices Billed to Germany from the year '2010' to '2012' in the increasing order of Invoice date.
>
> *Note*: SQLite doesn't support the `YEAR` function. Instead use `strftime("%Y", Invoice.InvoiceDate)` to extract the year from the column `InvoiceDate` as a string. [Learn more.](https://www.w3resource.com/sqlite/sqlite-strftime.php)

![](https://i.imgur.com/LlR7aMA.png)

In [ ]:
%%sql

???

In [ ]:
# DON'T MODIFY THIS CELL! IT IS USED FOR EVALUATION.
ans4 = _

## Group By & Having

> **Question 5**: Write a SQL query to display Albums from the genres 'Rock' and 'Jazz' containing more than 10 tracks. Show the total number of tracks, Album Id, Album Title, and the Genre for the albums that fall in above criteria.

![](https://i.imgur.com/LHU0iWJ.png)

In [ ]:
%%sql

???

In [ ]:
# DON'T MODIFY OR MOVE THIS CELL! IT IS USED FOR EVALUATION.
ans5 = _

## Aggregate Functions



>  **Question 6**: Write a SQL query to display each invoice amount as the fraction of the total sales from the billing city. Show the InvoiceId, billing city name, invoice total, and invoice total as a fraction of city total. Round all numbers to 2 decimal places.



![](https://i.imgur.com/dAXN3oZ.png)

In [ ]:
%%sql

???

In [ ]:
# DON'T MODIFY OR MOVE THIS CELL! IT IS USED FOR EVALUATION.
ans6 = _

> **QUESTION 7**: Write a SQL query to calculate and display the running total of song seconds by album. Output should contain song name, albumid, album name, song seconds & the running total.



![](https://i.imgur.com/pTXA1Wc.png)

In [ ]:
%%sql

???

In [ ]:
# DON'T MODIFY OR MOVE THIS CELL! IT IS USED FOR EVALUATION.
ans7 = _

## Value Functions

> **QUESTION 8**: Write a SQL query to calculate the monthly sales and percent change in sales for each month. Round all numbers to 2 decimal places.



*Hint:* Use Invoice table



![](https://i.imgur.com/Kvm90mU.png)

In [ ]:
%%sql

???

In [ ]:
# DON'T MODIFY OR MOVE THIS CELL! IT IS USED FOR EVALUATION.
ans8 = _

## Advanced SQL Clauses

> **QUESTION 9**: Write a SQL query to calculate the average payment made by each customer. The result should contain top 10 customers with First Name, Last Name, Country & Average Payment. Round all numbers to 2 decimal places.

*Hint*: Create a CTE with the name `AvgPayment` to write the above query


![](https://i.imgur.com/cqrpBtm.png)

In [ ]:
%%sql

???

In [ ]:
# DON'T MODIFY OR MOVE THIS CELL! IT IS USED FOR EVALUATION.
ans9 = _

>  **QUESTION 10**: Write a SQL query to calculate & display the year-wise total sales in the countries Ireland, Netherlands & Finland countries. Round all numbers to 2 decimal places.



![](https://i.imgur.com/D2Qbh3s.png)

In [ ]:
%%sql

???

In [ ]:
# DON'T MODIFY OR MOVE THIS CELL! IT IS USED FOR EVALUATION.
ans10 = _

## LeetCode Problems

### [Problem 1672](https://leetcode.com/problems/richest-customer-wealth/) - Richest Customer Wealth

You are given an `m x n` integer grid `accounts` where `accounts[i][j]` is the amount of money the `i​​​​​​​​​​​th​​​​` customer has in the `j​​​​​​​​​​​th​​​​` bank. Return the <b>*wealth*</b> that the richest customer has.

A customer's **wealth** is the amount of money they have in all their bank accounts. The richest customer is the customer that has the maximum **wealth.**



Example 1:
```
Input: accounts = [[1,2,3],[3,2,1]]
Output: 6
Explanation:
1st customer has wealth = 1 + 2 + 3 = 6
2nd customer has wealth = 3 + 2 + 1 = 6
Both customers are considered the richest with a wealth of 6 each, so return 6.
```
Example 2:
```
Input: accounts = [[1,5],[7,3],[3,5]]
Output: 10
Explanation:
1st customer has wealth = 6
2nd customer has wealth = 10
3rd customer has wealth = 8
The 2nd customer is the richest with a wealth of 10.
```
Example 3:
```
Input: accounts = [[2,8,7],[7,1,3],[1,9,5]]
Output: 17
```

Constraints:
```
m == accounts.length
n == accounts[i].length
1 <= m, n <= 50
1 <= accounts[i][j] <= 100
```

In [ ]:
def maximumWealth(accounts):
    ???

In [ ]:
# This should return "True"
maximumWealth([[1,2,3],[3,2,1]]) == 6

In [ ]:
# This should return "True"
maximumWealth([[1,5],[7,3],[3,5]]) == 10

In [ ]:
# This should return "True"
maximumWealth([[2,8,7],[7,1,3],[1,9,5]]) == 17

### [Problem 371](https://leetcode.com/problems/sum-of-two-integers/) - Sum of Two Integers

Given two integers `a` and `b`, return the *sum of the two integers without using the operators `+` and `-`.*

Example 1:
```
Input: a = 1, b = 2
Output: 3
```
Example 2:
```
Input: a = 2, b = 3
Output: 5
```

Constraints:
```
-1000 <= a, b <= 1000
```

In [ ]:
def getSum(a, b):
    ???

In [ ]:
# This should return "True"
getSum(1,2) == 3

In [ ]:
# This should return "True"
getSum(2,3) == 5

## Make a Submission

To make a submission, you can submit your Jovian notebook link on the assignment page.

You can make any number of submissions. Only your final submission will be considered for grading.

Here are some more optional questions you can try solving: https://github.com/LucasMcL/15-sql_queries_02-chinook